In [1]:
import pandas as pd
import numpy as np
import re
import requests
import bs4
import lxml
import plotly.express as px
import time
from playwright.sync_api import sync_playwright
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

pd.options.plotting.backend = 'plotly'

In [108]:
driver = webdriver.Chrome()
url = r'https://www.nba.com/stats/players/advanced?SeasonType=Regular+Season'
driver.get(url)

def select_by_index(xpath, index):
    """Select option by index from a dropdown identified by XPath."""
    try:
        select_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        select = Select(select_element)
        select.select_by_index(index)
    except TimeoutException:
        print(f"Timed out waiting for element located by {xpath}.")

select_by_index('/html/body/div[1]/div[2]/div[2]/div[3]/section[1]/div/div/div[1]/label/div/select', 1)

select_by_index("/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select", 0)


try:    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'Crom_table__p1iZz'))
    )
    src = driver.page_source
    parser = bs4.BeautifulSoup(src, 'html.parser')
    table = parser.find('table', attrs={'class': 'Crom_table__p1iZz'})
    tt = table.find('tbody', attrs={'class':'Crom_body__UYOcU'})
    players = tt.find_all('tr')
    playerList = [p.find_all('td')[1] for p in players]
    playerList = [p.find('a') for p in players]
    names = [a.text for a in playerList]
    links = [a.get('href') for a in playerList]
    df = pd.DataFrame(index=names, data = {'url':links})

except TimeoutException:
    print("Timed out waiting for table to be present.")
    
driver.close()

In [109]:
df

,url
Mikal Bridges,/stats/player/1628969
Derrick White,/stats/player/1628401
Harrison Barnes,/stats/player/203084
Isaiah Hartenstein,/stats/player/1628392
Jordan Poole,/stats/player/1629673
...,...
Jacob Gilyard,/stats/player/1631367
Louis King,/stats/player/1629663
Michael Foster Jr.,/stats/player/1630701
RaiQuan Gray,/stats/player/1630564


In [110]:
driver = webdriver.Chrome()


statsDict = {
    'NAME' : [],
    'GAME' : [],
    'W/L' : [],
    'MINUTES' : [],
    'OFFRTG' : [],
    'DEFRTG' : [],
    'NETRTG' : [],
    'AST%' : [],
    'AST/TO' : [],
    'AST RATIO' : [],
    'OREB%' : [],
    'DREB%' : [],
    'REB%' : [],
    'TO RATIO' : [],
    'EFG%' : [],
    'TS%' : [],
    'USG%' : [],
    'PACE' : [],
    'PIE' : [], 
}
for i in range(0,df.shape[0]):
    playerURL = f'https://www.nba.com{df["url"].iloc[i]}/boxscores-advanced'
    driver.get(playerURL)
    
    try:
        alert = driver.switch_to.alert
        alert.dismiss()
    except:
        pass
    
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select")))
    except:
        try:
            driver.refresh()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select")))
        except:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select")))

    select = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select"))))
    options = select.options
    
    if any(option.text == '2022-23' for option in options):
        seaSelect = Select(driver.find_element('xpath',"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select"))
        seaSelect.select_by_value('2022-23')
        time.sleep(1)
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select")))
        except:
            try:
                driver.refresh()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select")))
            except:
                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select")))
            
        regSelect = Select(driver.find_element('xpath',r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select"))
        regSelect.select_by_value('Regular Season')
        time.sleep(1)

        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))        
            
        except:
                try:
                    driver.refresh()
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))
                except:
                    try:
                        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))
                    except:
                        continue
            
        src = driver.page_source
        table = bs4.BeautifulSoup(src, 'html.parser')
        statTable = table.find('tbody', attrs={'class': 'Crom_body__UYOcU'})
        games = statTable.find_all('tr')
        
        for game in games:
            stats = game.find_all('td')
            stats = [x.text for x in stats]
            statsDict['NAME'].append(df.index[i])
            statsDict['GAME'].append(stats[0])
            statsDict['W/L'].append(stats[1])
            statsDict['MINUTES'].append(float(stats[2]))
            statsDict['OFFRTG'].append(float(stats[3]))
            statsDict['DEFRTG'].append(float(stats[4]))
            statsDict['NETRTG'].append(float(stats[5]))
            statsDict['AST%'].append(float(stats[6]))
            statsDict['AST/TO'].append(float(stats[7]))
            statsDict['AST RATIO'].append(float(stats[8]))
            statsDict['OREB%'].append(float(stats[9]))
            statsDict['DREB%'].append(float(stats[10]))
            statsDict['REB%'].append(float(stats[11]))
            statsDict['TO RATIO'].append(float(stats[12]))
            statsDict['EFG%'].append(float(stats[13]))
            statsDict['TS%'].append(float(stats[14]))
            statsDict['USG%'].append(float(stats[15]))
            statsDict['PACE'].append(float(stats[16]))
            statsDict['PIE'].append(float(stats[17]))
    
statsDict

{'NAME': ['Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Derrick White',
  'Derrick White',
  'D

In [162]:
statDF = pd.DataFrame(data = statsDict)
statDF.to_csv('2022-2023-advanced.csv', index=False)

In [113]:
driver = webdriver.Chrome()

tradDict = {
    'NAME' : [],
    'GAME' : [],
    'W/L' : [],
    'MIN' : [],
    'PTS' : [],
    'FGM' : [],
    'FGA' : [],
    'FG%' : [],
    '3PM' : [],
    '3PA' : [],
    '3P%' : [],
    'FTM' : [],
    'FTA' : [],
    'FT%' : [],
    'OREB' : [],
    'DREB' : [],
    'REB' : [],
    'AST' : [],
    'STL' : [],
    'BLK' : [],
    'TOV' : [],
    'PF' : [],
    '+/-' : [],
}

for i in range(0,df.shape[0]):
    playerURL = f'https://www.nba.com{df["url"].iloc[i]}/boxscores-traditional'
    driver.get(playerURL)
    
    try:
        alert = driver.switch_to.alert
        alert.dismiss()
    except:
        pass
    
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select")))
    except:
        try:
            driver.refresh()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select")))
        except:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select")))

    select = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select"))))
    options = select.options
    
    if any(option.text == '2022-23' for option in options):
        seaSelect = Select(driver.find_element('xpath',"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[1]/label/div/select"))
        seaSelect.select_by_value('2022-23')
        time.sleep(1)
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select")))
        except:
            try:
                driver.refresh()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select")))
            except:
                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select")))
            
        regSelect = Select(driver.find_element('xpath',r"/html/body/div[1]/div[2]/div[2]/section/div[4]/section[2]/div/div/div[2]/label/div/select"))
        regSelect.select_by_value('Regular Season')
        time.sleep(1)

        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))        
        except:
                try:
                    driver.refresh()
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))
                except:
                    try:
                        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Crom_body__UYOcU")))
                    except:
                        continue
                    
        src = driver.page_source
        table = bs4.BeautifulSoup(src, 'html.parser')
        statTable = table.find('tbody', attrs={'class': 'Crom_body__UYOcU'})
        games = statTable.find_all('tr')
        
        for game in games:
            stats = game.find_all('td')
            stats = [x.text for x in stats]
            tradDict['NAME'].append(df.index[i])
            tradDict['GAME'].append(stats[0])
            tradDict['W/L'].append(stats[1])
            tradDict['MIN'].append((stats[2]))
            tradDict['PTS'].append(float(stats[3]))
            tradDict['FGM'].append(float(stats[4]))
            tradDict['FGA'].append(float(stats[5]))
            tradDict['FG%'].append(float(stats[6]))
            tradDict['3PM'].append(float(stats[7]))
            tradDict['3PA'].append(float(stats[8]))
            tradDict['3P%'].append(float(stats[9]))
            tradDict['FTM'].append(float(stats[10]))
            tradDict['FTA'].append(float(stats[11]))
            tradDict['FT%'].append(float(stats[12]))
            tradDict['OREB'].append(float(stats[13]))
            tradDict['DREB'].append(float(stats[14]))
            tradDict['REB'].append(float(stats[15]))
            tradDict['AST'].append(float(stats[16]))
            tradDict['STL'].append(float(stats[17]))
            tradDict['BLK'].append(float(stats[18]))
            tradDict['TOV'].append(float(stats[19]))
            tradDict['PF'].append(float(stats[20]))
            tradDict['+/-'].append(float(stats[21]))
                
driver.close()
tradDict

{'NAME': ['Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Mikal Bridges',
  'Derrick White',
  'Derrick White',
  'D

In [175]:
tradDF = pd.DataFrame(data = tradDict)
tradDF.to_csv('2022-2023-traditional.csv', index=False)
tradDF['MINUTES'] = tradDF['MIN'].apply(lambda x: round(float(x.replace(':', '.')),0))
tradDF = tradDF.drop(columns = ['MIN'])

In [176]:
tradDF

,NAME,GAME,W/L,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,MINUTES
0,Mikal Bridges,"Apr 09, 2023 - BKN vs. PHI",L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Mikal Bridges,"Apr 07, 2023 - BKN vs. ORL",W,22.0,7.0,24.0,29.2,1.0,12.0,8.3,...,0.0,5.0,5.0,2.0,2.0,0.0,3.0,1.0,14.0,36.0
2,Mikal Bridges,"Apr 05, 2023 - BKN @ DET",W,26.0,10.0,26.0,38.5,2.0,11.0,18.2,...,1.0,2.0,3.0,6.0,0.0,1.0,1.0,0.0,25.0,37.0
3,Mikal Bridges,"Apr 04, 2023 - BKN vs. MIN",L,24.0,9.0,24.0,37.5,1.0,5.0,20.0,...,0.0,5.0,5.0,2.0,2.0,1.0,3.0,2.0,-9.0,40.0
4,Mikal Bridges,"Apr 02, 2023 - BKN vs. UTA",W,30.0,9.0,25.0,36.0,1.0,8.0,12.5,...,0.0,7.0,7.0,2.0,3.0,0.0,1.0,2.0,8.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19771,Jacob Gilyard,"Apr 09, 2023 - MEM @ OKC",L,3.0,1.0,3.0,33.3,1.0,3.0,33.3,...,0.0,4.0,4.0,7.0,3.0,0.0,2.0,3.0,-13.0,40.0
19772,Louis King,"Apr 09, 2023 - PHI @ BKN",W,20.0,8.0,13.0,61.5,4.0,8.0,50.0,...,2.0,2.0,4.0,2.0,1.0,0.0,2.0,4.0,2.0,28.0
19773,Michael Foster Jr.,"Nov 22, 2022 - PHI vs. BKN",W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,1.0
19774,RaiQuan Gray,"Apr 09, 2023 - BKN vs. PHI",L,16.0,6.0,12.0,50.0,2.0,5.0,40.0,...,3.0,6.0,9.0,7.0,0.0,1.0,4.0,5.0,-14.0,35.0


In [178]:
statDF['GAME'] = statDF['GAME'].apply(lambda x: x.replace('vs.', '@'))
tradDF['GAME'] = tradDF['GAME'].apply(lambda x: x.replace('vs.', '@'))
tradDF = tradDF.rename(columns={'MIN':'MINUTES'})
mergeStats = statDF.merge(tradDF, how='inner', on=['NAME','GAME','W/L','MINUTES'])
mergeStats.to_csv('2022-2023-merged.csv', index=False)

In [180]:
GCODE = []
split = mergeStats['GAME'].apply(lambda x: x.split(' - '))
for i in split:
    codeSum = 0
    codeSumTwo = 0
    for x in i[0]:
        codeSum = codeSum + ord(x)
    for x in i[1]:
        codeSumTwo = codeSumTwo + ord(x)
    GCODE.append(str(codeSum) + str(codeSumTwo))
    
mergeStats['GCODE'] = GCODE
mergeStats['Team'] = mergeStats['GAME'].apply(lambda x: x.split(' - ')[1][0:3])
    

In [181]:
meanStats = mergeStats.select_dtypes(include='number')
meanStats['NAME'] = mergeStats['NAME']
meanStats = meanStats.groupby('NAME').mean()
meanCols = np.array(meanStats.columns, dtype='<U4')
string = ' AVG'
meanCols = np.core.defchararray.add(meanCols, string)
meanStats = meanStats.set_axis(meanCols, axis=1)
totalStats = mergeStats.merge(meanStats, how='inner', on='NAME')


In [183]:
totalStats.to_csv('2022-2023-total.csv', index=False)